## Линейная регрессия

В этом задании Вам предстоит: 

-познакомиться с предобработкой данных;

-найти МНК-решение аналитически;

-найти МНК-решение, оптимизируя функцию потерь;

-обучить библиотечный алгоритм линейной регрессии;

-применить регуляризацию и подобрать лучший параметр для неё;

-познакомиться с PCA

In [1]:
import pandas as pd
import numpy as np
import scipy as sc
from sklearn import preprocessing, cross_validation,  linear_model,  metrics,  pipeline, grid_search
import seaborn as sns
from matplotlib import pyplot as plt

In [4]:
%matplotlib inline

## Считываем данные из репозитория UCI

In [5]:
url='http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
data=pd.read_csv(url, sep=',' , header=None, na_values='?')    

1) Ознакомьтесь с данными, посмотрите на размер таблицы


2) Озаглавьте столбцы

3) Посмотрите, есть ли  в последнем столбце (а это наш целевой вектор y) пропуски. Если их немного, просто удалите строки с пропусками

4) Создайте вектор y (это последний столбец data), удалите этот столбец из таблицы

5) Посмотрите на взаимосвязь признаков с помощью [sns.pairplot](http://seaborn.pydata.org/generated/seaborn.pairplot.html?highlight=pairplot#seaborn.pairplot)

## Предобработка категориальных, количественных и бинарных признаков проходит по-разному. Разделим их, а после обработки соединим снова.

6)  Создайте 3 списка из имён признаков: категориальные, количественные и бинарные (см. демонстрационные файлы)

7) Проверьте, если пропуски в данных

8) Заполните пропуски в бинарных признаках. Например, следующим способом 

In [ ]:
data_bin=data[bin_cols].describe()

for c in bin_cols:
    top = data_bin[c]['top']
    top_items = data[c] == top
    data.loc[top_items, c] = 0
    data.loc[np.logical_not(top_items), c] = 1

9) Заполните пропуски в количественных признаках. Например, следующим способом 

In [18]:
data_real=data[real_cols].dropna().describe()

for c in real_cols:
    data[c].fillna(data_real[c].quantile(),inplace=True)

## Закодируем катигориальные признаки, используя DictVectorizer (см. демонстрационные файлы)

10) Запишите закодированные признаки в отдельный массив

## Разделим выборку на обучающую и контрольную

11) Разделите выборку с помощью **train_test_split** с размером тестовой выборки 0.3

## Стандартизируем количественные признаки

12) Используйте **sklearn.preprocessing.StandardScaler**, запишите стандартизированные данные в новые массивы

**NB** Можете попробовать другие методы. Например, **sklearn.preprocessing.normalize**

## Соедините массивы признаков

13) Создайте массивы X_train и  X_test (например, используя np.hstack) 

## Нахождение решения аналитически

14) Найдите решение нормального уравнения МНК   $F^TF\alpha=F^Ty$ с помощью [scipy.linalg](https://docs.scipy.org/doc/scipy-0.18.1/reference/linalg.html)

15) Найдите минимум функционала ошибки $||F\alpha-y||^2$ с помощью [scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)

## Обучите линейную регрессию

16) Посмотрите на метрики качества $R^2 $ и MSE

## Посмотрите на коэффициенты, на корреляцию признаков. Присутствует ли мультиколлинеарность? 

17) Посмотрите на получившиеся коэффициенты,соответствующие признакам, используя [zip](https://docs.python.org/2/library/functions.html#zip)

18) **Для количественных признаков:** посмотрите на корреляцию [pandas.DataFrame.corr()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html)


**Для категориальных признаков:** вспомните проверку независимости признаков с помощью критерия хи-квадрат.

Для столбцов data с категориальными признаками напишите функцию, которая возвращает результат применения  критерия хи-квадрат для таблиц сопряженности. Составьте таблицу сопряженности для каждой пары категориальных признаков (используйте для этого, например, комбинацию **pandas.DataFrame.groupby(['column1', 'column2']).size()** и **pandas.DataFrame.unstack('column1')**. Для полученной таблицы используйте **scipy.stats.chi2_contingency**.

Посмотрите на  p-значение для каждой пары признаков. Есть ли связь между признаками?

**Для проверки наличия связи между каждой парой категориальныx и количественныx признаков** используйте [ANOVA](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html). Также визуализируйте зависимости с помощью [sns.boxplot](http://seaborn.pydata.org/generated/seaborn.boxplot.html?highlight=boxplot#seaborn.boxplot)

## Примените регуляризацию, посмотрите, изменилось ли качество

19) Обучите Lasso с параметрами по умолчанию, посмотрите на r2 и MSE на тестовой выборке

20) Обучите Ridge с параметрами по умолчанию, посмотрите на r2 и MSE на тестовой выборке

## Подбор параметра регуляризации

In [3]:
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV

21) С помощью LassoCV подберите лучший параметр  регуляризации из диапазона [1,100] с шагом 5 и постройте график зависимости среднего по фолдам значения MSE от параметра регуляризации (см. демонстрационные файлы). ПО умолчанию используется 3-fold CV.

Посмотрите на r2 и MSE на тестовой выборке

22) Подберите лучший параметр регуляризации из диапазона [1,100] с шагом 5 с помощью RidgeCV.  
Имейте в виду, что значения функционала качества на кросс-валидации $cv\_values\_ \ $   доступны, если используется стратегия Leave-One-Out.

Посмотрите на r2 и MSE на тестовой выборке

23) Посмотрите на коэффициенты Lasso и Ridge при лучшем значении  параметра. В чём разница?

## PCA

Познакомимся с Pipeline. Pipeline позволяет создать модель из несколько последовательных шагов, параметры для этих шагов могут подбираться совместно по кросс-валидации: http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

Например, если нам нужно стандартизировать признаки, а затем оценить качество по кросс-валидации, то нужно проводить стандартизацию контрольной и тестовой частей для каждого фолда отдельно. Если мы применим стандартизацию ко всей выборке, а затем проведём кросс-валидацию, то оценки получатся смещёнными.

24) Применим Pipeline для выбора числа главных компонент

In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression


pipe = Pipeline([
    ('reduce_dim', PCA()),
    ('classify', LinearRegression())
])

N_FEATURES_OPTIONS = np.arange(30)[1:]

param_grid = {
        'reduce_dim__n_components': N_FEATURES_OPTIONS,
    }

grid = grid_search.GridSearchCV(pipe, cv=3, n_jobs=2, param_grid=param_grid)
grid.fit(X_train, y_train)

grid.best_params_
grid.best_score_